# Projekt ML, przygotowanie danych, część 2.

Poniższy rozdział skupia się na przygotowaniu danych w sensie analizy cech zawartych w danych.

Uczenie maszynowe zrealizowano w pliku "Projekt ML Machine Learning"

Do analizy przyjęto dane ze zbiotu 'cars.csv'

Zbiór danych z 392 obserwacjami i 9 zmiennymi:

• mpg - miles per gallon - zużycie paliwa (w milach na galon paliwa)

• cylinders - liczba cylindrów (od 4 do 8)

• displacement - objętość skokowa (cale sześcienne)

• horsepower - moc (konie mechaniczne)

• weight - waga (w funtach)

• acceleration - przyspieszenie w sekundach do 60 mil na godzinę

• year - rok produkcji (z ostatnie cyfry roku, XX wiek)

• origin - kraj produkcji (1. American, 2. European, 3. Japanese)

• name - nazwa samochodu

## Cel analizy: Od czego zależy bardziej ekonomiczne zużycie paliwa? Czy da się przewidzieć zużycie paliwa na podstawie danych ze zbioru?

# 1. Wstępne przetwarzanie danych

- wczytanie
- sprawdzenie typów danych
- sprawdzenie co jest w danych
- sprawdzenie błędów w danych

In [24]:
# import potrzebnych bibliotek
import numpy as np
import pandas as pd
import seaborn as sns

In [25]:
# wczytanie danych z pliku
df_raw = pd.read_csv('cars.csv')
# podstawowe informacje o danych
df_raw.info()
df_raw.head()
df_raw.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    int64  
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   year          392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 27.7+ KB


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,1.576531
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,0.805518
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,1.000000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


In [26]:
# sprawdzenie, czy są jakieś braki w danych
df_raw.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

W zbiorze nie ma braków danych. Na podstawie opisu danych sprawdzam wyniki polecenia .describe() . Na tej podstawie stwierdzam, że należy sprawdzić dane dla cylinders == 3. Inne dane wstępnie wyglądają na poprawne: są większe od zera, ich zakresy wyglądają na poprawne.

In [27]:
# sprawdzenie ilości rekordów dla cylinders==4
df_raw[df_raw['cylinders'] == 3].shape[0]
df_raw[df_raw['cylinders'] == 3].head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
70,19.0,3,70.0,97,2330,13.5,72,3,mazda rx2 coupe
110,18.0,3,70.0,90,2124,13.5,73,3,maxda rx3
241,21.5,3,80.0,110,2720,13.5,77,3,mazda rx-4
331,23.7,3,70.0,100,2420,12.5,80,3,mazda rx-7 gs


Ze względu na niewielką ilość niepoprawnych danych w dalszej analizie pomijam wartości dla cylinders==3

In [28]:
# dane bez wartości cylinders==3
df_raw = df_raw[df_raw['cylinders'] != 3]
df_raw.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
count,388.000000,388.000000,388.000000,388.000000,388.000000,388.000000,388.000000,388.000000
mean,23.475773,5.497423,195.668814,104.523196,2983.554124,15.564948,75.984536,1.561856
std,7.836404,1.695866,104.442110,38.678987,851.448791,2.762832,3.688066,0.796497
min,9.000000,4.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.900000,73.000000,1.000000
50%,23.000000,4.000000,151.000000,92.500000,2822.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,302.000000,129.000000,3622.500000,17.125000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


# Charakterystyka zbioru danych

• mpg - miles per gallon - dane liczbowe ciągłe

• cylinders - dane liczbowe całkowite, mogą być traktowane jako kategoria

• displacement - dane liczbowe ciągłe

• horsepower - dane liczbowe ciągłe

• weight - dane liczbowe ciągłe

• acceleration - dane liczbowe ciągłe

• year - dane liczbowe całkowite, mogą być traktowane jako kategoria

• origin - dane liczbowe całkowite, mogą być traktowane jako kategoria

• name - nazwa samochodu - dane tekstowe, opisowe. Może ktoś wpadnie na pomysł, aby ich użyć do ML :)

Dane mają prawidłowe typy danych w kolumnach, nie ma brakujących wartości, podstawowe nieprawidłowości usunięte.
Wstępnie przetworzone dane zapisuję do pliku 'cars_preprocessed.csv'

In [29]:
# Zapisanie wstępnie przetworzonych danych do pliku
df_raw.to_csv('cars_preprocessed_2.csv', index=False)

# 2. Przygotowanie zbioru danych do analizy przy pomocy uczenia maszynowego

Na potrzeby uczenia maszynowego przygotuję 4 typy zbiorów danych:

2.1. Dane kategoryczne zostaną potraktowane jako liczbowe, kolumna 'names' nie bierze udziału w ML

2.2. Dane kategoryczne 'origin' rozdzielone (dummies), reszta danych jako dane liczbowe, kolumna 'names' nie bierze udziału w ML

2.3. Dane kategoryczne 'origin' i 'cylinders' rozdzielone (dummies), reszta danych jako dane liczbowe, kolumna 'names' nie bierze udziału w ML

2.4. Dane kategoryczne 'origin' i 'cylinders' rozdzielone (dummies), reszta danych jako dane liczbowe, kolumna 'names' zostaje rozpisana na słownik i bierze udział w ML  (taka namiastka LLM, to jest dobry moment na uśmiech)


Dla każdego przypadku zmienne odstające (outliers) zostaną osobno określone.



# 2.1 Dane kategoryczne potraktowane jako liczbowe.

Kolumna 'names' nie bierze udziału w ML, ale pozostaje w danych

In [30]:
# wczytanie danych z pliku
df = pd.read_csv('cars_preprocessed_2.csv')

Wykrywanie wartości odstających (outliers).

Wykorzystałem metodę z Q1, Q3, IQR * 1.5

In [31]:
# w DataFrame df_outliers zawarte są wartości bool wskazujące na wykrycie outliers
# nazwy kolumn analogicznie jak w danych źródłowych
df_outliers = pd.DataFrame()
df_numeric = df.iloc[:, 0:8]    # w danych numerycznych pomijam kolumnę names 

for col in df_numeric.columns:
    # obliczenie Q1, Q3, IQR
    Q1 = df_numeric[col].quantile(0.25)
    Q3 = df_numeric[col].quantile(0.75)
    IQR = Q3 - Q1
    
    # wykrycie outliers
    outliers = (df_numeric[col] < (Q1 - 1.5 * IQR)) | (df_numeric[col] > (Q3 + 1.5 * IQR))
    
    # dodanie kolumny do DataFrame df_outliers
    df_outliers[col] = outliers     # zawartość pętli for napisał czat :)

df_outliers['is_outlier'] = df_outliers.any(axis=1)

df_numeric.shape
df_outliers.shape
df_outliers.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,is_outlier
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False


In [32]:
# sprawdzenie ilości wykrytych outliers
df_outliers['is_outlier'].sum()

np.int64(17)

W tym przypadku zbiorze 'cars_preprocessed.csv' wykryto 17 wartości odstających (outliers).

In [33]:
# usunięcie outliers z danych
df_no_outl = df[~df_outliers['is_outlier']].copy()
df_no_outl.shape

(371, 9)

In [34]:
# zapisanie danych do pliku
df_no_outl.to_csv('cars_4_ML_no_dummy_cleared.csv', index=False)

## 2.2. Dane kategoryczne 'origin' rozdzielone (dummies), reszta danych jako dane liczbowe

Kolumna 'names' nie bierze udziału w ML

In [35]:
# wczytanie danych z pliku
df = pd.read_csv('cars_preprocessed.csv')

In [36]:
# przygotowanie danych, rozpisanie 'origin' jako dummies
df_origin_dummy = pd.get_dummies(df['origin'], prefix='origin')
df_dummy = pd.concat([df, df_origin_dummy], axis=1)
df_dummy = df_dummy.drop(columns=['origin'])  # usunięcie kolumny 'origin'
df_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           388 non-null    float64
 1   cylinders     388 non-null    int64  
 2   displacement  388 non-null    float64
 3   horsepower    388 non-null    int64  
 4   weight        388 non-null    int64  
 5   acceleration  388 non-null    float64
 6   year          388 non-null    int64  
 7   name          388 non-null    object 
 8   origin_1      388 non-null    bool   
 9   origin_2      388 non-null    bool   
 10  origin_3      388 non-null    bool   
dtypes: bool(3), float64(3), int64(4), object(1)
memory usage: 25.5+ KB


Wykrywanie wartości odstających (outliers). Dla każdej kategorii w 'origin' dane odstające są osobno sa wykrywane.

Wykorzystałem metodę z Q1, Q3, IQR * 1.5

In [37]:
# w DataFrame df_outliers zawarte są wartości bool wskazujące na wykrycie outliers
# nazwy kolumn analogicznie jak w danych źródłowych z uwzględnieniem kategorii, dla której zostały policzone
#  w danych pomijam kolumnę names 

df_outliers = pd.DataFrame()
df_numeric = df_dummy.iloc[:, 0:7]    
df_categorical = df_dummy.iloc[:, 8:]  # 'origin' jako dane kategoryczne

for col_num in df_numeric.columns:
    for col_cat in df_categorical.columns:
        filter = df_dummy[col_cat] == 1 
        Q1 = df_dummy.loc[filter, col_num].quantile(0.25)
        Q3 = df_dummy.loc[filter, col_num].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((df_dummy[col_num] < (Q1 - 1.5 * IQR )) & filter  ) | ((df_dummy[col_num] > (Q3 + 1.5 * IQR)) & filter)
        col_name = col_num+ '_' + col_cat
        df_outliers[col_name] = outliers
        print(col_name, '  ilość outliers:', outliers.sum())

df_outliers['is_outlier'] = df_outliers.any(axis=1)

print('--------------------------------------------------')
print('Łączna ilość wykrytych outliers:', df_outliers['is_outlier'].sum())


mpg_origin_1   ilość outliers: 3
mpg_origin_2   ilość outliers: 5
mpg_origin_3   ilość outliers: 1
cylinders_origin_1   ilość outliers: 0
cylinders_origin_2   ilość outliers: 7
cylinders_origin_3   ilość outliers: 6
displacement_origin_1   ilość outliers: 0
displacement_origin_2   ilość outliers: 4
displacement_origin_3   ilość outliers: 2
horsepower_origin_1   ilość outliers: 0
horsepower_origin_2   ilość outliers: 2
horsepower_origin_3   ilość outliers: 0
weight_origin_1   ilość outliers: 0
weight_origin_2   ilość outliers: 0
weight_origin_3   ilość outliers: 0
acceleration_origin_1   ilość outliers: 0
acceleration_origin_2   ilość outliers: 0
acceleration_origin_3   ilość outliers: 0
year_origin_1   ilość outliers: 0
year_origin_2   ilość outliers: 0
year_origin_3   ilość outliers: 0
--------------------------------------------------
Łączna ilość wykrytych outliers: 22


W tym przypadku zbiorze 'cars_preprocessed.csv' wykryto 22 wartości odstające (outliers).

In [38]:
# usunięcie outliers z danych
df_dummy_no_outl = df_dummy[~df_outliers['is_outlier']].copy()
df_dummy_no_outl.shape

(366, 11)

In [39]:
# zapisanie danych do pliku
df_dummy_no_outl.to_csv('cars_4_ML_dummy_origin_cleared.csv', index=False)

## 2.3. Dane kategoryczne 'origin' i 'cylinders' rozdzielone (dummies), reszta danych jako dane liczbowe.

Kolumna 'names' nie bierze udziału w ML

In [40]:
# wczytanie danych z pliku
df = pd.read_csv('cars_preprocessed.csv')

In [41]:
# przygotowanie danych, rozpisanie 'origin' oraz 'cylinders' jako dummies
df_origin_dummy = pd.get_dummies(df['origin'], prefix='origin')
df_cylinders_dummy = pd.get_dummies(df['cylinders'], prefix='cylinders')
df_dummy = pd.concat([df, df_origin_dummy, df_cylinders_dummy], axis=1)
df_dummy = df_dummy.drop(columns=['origin', 'cylinders'])  # usunięcie kolumny 'origin' i 'cylinders'
df_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           388 non-null    float64
 1   displacement  388 non-null    float64
 2   horsepower    388 non-null    int64  
 3   weight        388 non-null    int64  
 4   acceleration  388 non-null    float64
 5   year          388 non-null    int64  
 6   name          388 non-null    object 
 7   origin_1      388 non-null    bool   
 8   origin_2      388 non-null    bool   
 9   origin_3      388 non-null    bool   
 10  cylinders_4   388 non-null    bool   
 11  cylinders_5   388 non-null    bool   
 12  cylinders_6   388 non-null    bool   
 13  cylinders_8   388 non-null    bool   
dtypes: bool(7), float64(3), int64(3), object(1)
memory usage: 24.0+ KB


Wykrywanie wartości odstających (outliers). Dla każdej kategorii w 'origin' oraz 'cylinders' dane odstające są osobno sa wykrywane.

Wykorzystałem metodę z Q1, Q3, IQR * 1.5

In [42]:
# w DataFrame df_outliers zawarte są wartości bool wskazujące na wykrycie outliers
# nazwy kolumn analogicznie jak w danych źródłowych z uwzględnieniem kategorii, dla której zostały policzone
#  w danych pomijam kolumnę names 

df_outliers = pd.DataFrame()
df_numeric = df_dummy.iloc[:, 0:6]    
df_categorical = df_dummy.iloc[:, 7:]  # 'origin' oraz 'cylinders' jako dane kategoryczne

for col_num in df_numeric.columns:
    for col_cat in df_categorical.columns:
        filter = df_dummy[col_cat] == 1 
        Q1 = df_dummy.loc[filter, col_num].quantile(0.25)
        Q3 = df_dummy.loc[filter, col_num].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((df_dummy[col_num] < (Q1 - 1.5 * IQR )) & filter  ) | ((df_dummy[col_num] > (Q3 + 1.5 * IQR)) & filter)
        col_name = col_num+ '_' + col_cat
        df_outliers[col_name] = outliers
        #print(col_name, '  ilość outliers:', outliers.sum())

df_outliers['is_outlier'] = df_outliers.any(axis=1)

print('--------------------------------------------------')
print('Łączna ilość wykrytych outliers:', df_outliers['is_outlier'].sum())


--------------------------------------------------
Łączna ilość wykrytych outliers: 31


W tym przypadku zbiorze 'cars_preprocessed.csv' wykryto 31 wartości odstających (outliers).

In [43]:
# usunięcie outliers z danych
df_dummy_all_no_outl = df_dummy[~df_outliers['is_outlier']].copy()
df_dummy_all_no_outl.shape

(357, 14)

In [44]:
# zapisanie danych do pliku
df_dummy_all_no_outl.to_csv('cars_4_ML_dummy_all_cleared.csv', index=False)

## 2.4. Dane kategoryczne 'origin' i 'cylinders' rozdzielone (dummies), reszta danych jako dane liczbowe, kolumna 'names' zostaje rozpisana na słownik i bierze udział w ML 

Uzupełnienie danych o słownik na podstawie kolumny 'names'. Dane słownikowe w postaci kategorii (dummies).

In [45]:
feature_str_list = []
for i in df_dummy_all_no_outl['name'].tolist():
    i_split = i.split(' ')
    for i1 in i_split:
        feature_str_list = feature_str_list + [i1]
        
feature_set = set(feature_str_list)
print('Ilość unikalnych cech w kolumnie names:', len(feature_set))

df_dummy_all_no_outl_LLM = df_dummy_all_no_outl.copy()
names = df_dummy_all_no_outl['name'].to_numpy(dtype=str).copy()

# wyłączenie komunikatów warnings
# w tym przypadku poniższy kod generuje macierz rzadką
import warnings
warnings.simplefilter(action='ignore', category=Warning)

for feature in feature_set:
    col_name = 'name_' + feature
    #for name in np.nditer(names):
    df_dummy_all_no_outl_LLM[col_name] = np.strings.count(names, feature, start=0, end=None).reshape((-1,))

    #print(temp)
    #assert 0, 'Przerwano wykonywanie'
            
#print(df_dummy_all_no_outl_LLM.info())
#print(df_dummy_all_no_outl_LLM.head())

Ilość unikalnych cech w kolumnie names: 287


In [46]:
# zapisanie danych do pliku
df_dummy_all_no_outl_LLM.to_csv('cars_4_ML_dummy_all_LLM_cleared.csv', index=False)